In [1]:
%cd ../

/home/erland.fuadi/Python Project/supertrainer


In [2]:
from dotenv import load_dotenv
from hydra import compose, initialize
from omegaconf import OmegaConf

from supertrainer import StrictDict
from supertrainer.utils.helpers import login_hf

load_dotenv()
login_hf()

with initialize(config_path="../configs", version_base=None):
    cfg = compose(config_name="train", overrides=["+experiment=train_llm"])
    cfg = OmegaConf.to_container(cfg, resolve=True)
    cfg = StrictDict(cfg)
    print(cfg)

2024-09-17 18:52:05.873 | DEBUG    | supertrainer.utils.helpers:login_hf:18 - Use token from environment variable HUGGINGFACE_API_KEY


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/erland.fuadi/.cache/huggingface/token
Login successful
{'trainer': {'model_kwargs': {'attn_implementation': 'flash_attention_2', 'torch_dtype': 'auto', 'use_cache': False, 'device_map': None}, 'bitsandbytes_kwargs': {'load_in_4bit': True, 'bnb_4bit_quant_type': 'nf4', 'bnb_4bit_compute_dtype': None}, 'training_kwargs': {'gradient_accumulation_steps': 64, 'warmup_steps': 5, 'num_train_epochs': 3, 'learning_rate': 2e-05, 'fp16': False, 'bf16': True, 'logging_steps': 20, 'logging_strategy': 'steps', 'optim': 'adamw_8bit', 'weight_decay': 0.01, 'lr_scheduler_type': 'linear', 'seed': 3407, 'report_to': 'wandb', 'output_dir': './assets_local/smollm/outputs', 'load_best_model_at_end': 1, 'metric_for_

In [3]:
from supertrainer.utils.helpers import split_train_test_validation

split_dataset = split_train_test_validation(dataset, select_subset=0.1)

2024-09-17 17:19:08.517 | WARNING  | supertrainer.utils.helpers:split_train_test_validation:199 - Dataset is a DatasetDict. We will you're going to use the 'train' dataset.


In [7]:
from dotenv import load_dotenv
import os

load_dotenv()


split_dataset.push_to_hub("Erland/llm_sample_dataset", private=True, token=os.getenv("HUGGINGFACE_API_TOKEN"))

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Erland/llm_sample_dataset/commit/bc90b7fd6aa755a61f6d403533e86f91417e5b81', commit_message='Upload dataset', commit_description='', oid='bc90b7fd6aa755a61f6d403533e86f91417e5b81', pr_url=None, pr_revision=None, pr_num=None)

In [3]:
from supertrainer.data.llm import ConversationLLMDataset

dataset = ConversationLLMDataset(config=cfg["dataset"], is_testing=True)
dataset

In [4]:
formatted_dataset = dataset.prepare_dataset()
formatted_dataset

2024-09-17 18:52:10.611 | DEBUG    | supertrainer.data.llm:prepare_dataset:195 - Preparing dataset
2024-09-17 18:52:22.821 | DEBUG    | supertrainer.data.llm:prepare_dataset:204 - Applying formatting prompts


Map:   0%|          | 0/199 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/63 [00:00<?, ? examples/s]

2024-09-17 18:52:23.590 | DEBUG    | supertrainer.data.llm:formatting_prompt_func:163 - Remove assistant output from test datasetinstead, add the generation prompt
2024-09-17 18:52:23.590 | DEBUG    | supertrainer.data.llm:formatting_prompt_func:163 - Remove assistant output from test datasetinstead, add the generation prompt
2024-09-17 18:52:23.591 | DEBUG    | supertrainer.data.llm:formatting_prompt_func:163 - Remove assistant output from test datasetinstead, add the generation prompt
2024-09-17 18:52:23.591 | DEBUG    | supertrainer.data.llm:formatting_prompt_func:163 - Remove assistant output from test datasetinstead, add the generation prompt
2024-09-17 18:52:23.591 | DEBUG    | supertrainer.data.llm:formatting_prompt_func:163 - Remove assistant output from test datasetinstead, add the generation prompt
2024-09-17 18:52:23.592 | DEBUG    | supertrainer.data.llm:formatting_prompt_func:163 - Remove assistant output from test datasetinstead, add the generation prompt
2024-09-17 18:52

*** Example from the dataset ***
Example 1:
Substring not found.
--------------------
Example 2:
Substring not found.
--------------------
Example 3:
Substring not found.
--------------------
Example 4:
Substring not found.
--------------------
Example 5:
Substring not found.
--------------------


DatasetDict({
    train: Dataset({
        features: ['conversations', 'text'],
        num_rows: 199
    })
    validation: Dataset({
        features: ['conversations', 'text'],
        num_rows: 50
    })
    test: Dataset({
        features: ['conversations', 'text'],
        num_rows: 63
    })
})

In [7]:
formatted_dataset["train"]["text"][0]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nNiccolo Govender, a charismatic figure, is a master of charm and manipulation. He strides through life with a confident grace, leaving those around him in awe of his wealth and power. His jealousy and arrogance often manifest as sarcasm, aimed at those he deems inferior. A bully at heart, Niccolo seeks to dominate any situation, his rude behavior leaving those around him reeling. Despite his callous exterior, he remains fiercely loyal to those closest to him. He has a soft spot for you yet because you both hate each other he masks it by being as mean as he can with you.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n*You both are popular but hate eachother. He is the football captain and you are the captian of cheerleaders (what a coincidence lol)\nYou really dont like eachother but one day theres a Halloween party. You are in your house getting r

In [10]:
dataset.dataset["train"][0]["conversations"]

[{'from': 'system',
  'value': 'Niccolo Govender, a charismatic figure, is a master of charm and manipulation. He strides through life with a confident grace, leaving those around him in awe of his wealth and power. His jealousy and arrogance often manifest as sarcasm, aimed at those he deems inferior. A bully at heart, Niccolo seeks to dominate any situation, his rude behavior leaving those around him reeling. Despite his callous exterior, he remains fiercely loyal to those closest to him. He has a soft spot for you yet because you both hate each other he masks it by being as mean as he can with you.'},
 {'from': 'gpt',
  'value': '*You both are popular but hate eachother. He is the football captain and you are the captian of cheerleaders (what a coincidence lol)\nYou really dont like eachother but one day theres a Halloween party. You are in your house getting ready*'},
 {'from': 'human',
  'value': '*I was in my room getting ready , I dressed up to a black short tight dress , and I 

In [8]:
formatted_dataset["train"]["conversations"][0]

[{'from': 'system',
  'value': 'Niccolo Govender, a charismatic figure, is a master of charm and manipulation. He strides through life with a confident grace, leaving those around him in awe of his wealth and power. His jealousy and arrogance often manifest as sarcasm, aimed at those he deems inferior. A bully at heart, Niccolo seeks to dominate any situation, his rude behavior leaving those around him reeling. Despite his callous exterior, he remains fiercely loyal to those closest to him. He has a soft spot for you yet because you both hate each other he masks it by being as mean as he can with you.'},
 {'from': 'gpt',
  'value': '*You both are popular but hate eachother. He is the football captain and you are the captian of cheerleaders (what a coincidence lol)\nYou really dont like eachother but one day theres a Halloween party. You are in your house getting ready*'},
 {'from': 'human',
  'value': '*I was in my room getting ready , I dressed up to a black short tight dress , and I 